In [1]:
import json
import os

timestamps = [i * 0.02 for i in range(1500 + 1)]
replace = {
    'Aee.': 'A.',
    'Bee.': 'B.',
    'See.': 'C.',
    'Dee.': 'D.',
}

In [2]:
with open('mallm.json') as fopen:
    d = json.load(fopen)
    
len(d)

6564

In [3]:
result = []

for i in range(len(d)):
    audio = os.path.join('mallm-v2', f'{i}.mp3')
    align = os.path.join('mallm-v2-alignment_alignment', f'{i}.json')
    
    if not os.path.exists(audio):
        continue
    
    with open(align) as fopen:
        align = json.load(fopen)
    
    merged_data = []
    for entry in align[1:]:
        if entry['text'] == '.' and merged_data:
            merged_data[-1]['text'] += '.'
            merged_data[-1]['end'] = entry['end']
        else:
            merged_data.append(entry)

    for i in range(len(merged_data)):
        merged_data[i]['text'] = replace.get(merged_data[i]['text'], merged_data[i]['text'])
        
    align = merged_data
        
    segments, temp = [], [align[0]]
    last_t = align[0]['end']
    for c in align[1:]:
        if (c['start'] - last_t) > 0.2:
            segments.append(temp)
            temp = []

        last_t = c['end']
        temp.append(c)

    if len(temp):
        segments.append(temp)

    ts = []
    for s in segments:
        start = min(timestamps, key=lambda t: abs(t - s[0]['start']))
        end = min(timestamps, key=lambda t: abs(t - s[-1]['end']))
        w = ' '.join([c['text'] for c in s])
        t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
        ts.append(t)

    ts = ''.join(ts)
    new_text = text = f"<|startoftranscript|><|ms|><|transcribe|>{ts}<|endoftext|>"

    result.append({
        'new_text': new_text,
        'audio_filename': audio,
    })

In [4]:
len(result)

2422

In [5]:
result[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.34|> Manakah antara berikut merupakan catatan kontra bagi Akaun Jualan?<|3.26|><|3.74|> A. Pulangan masuk. B. Pulangan keluar.<|6.50|><|6.76|> C. Angkutan masuk. D.<|8.30|><|8.56|> Angkutan keluar.<|9.36|><|endoftext|>',
 'audio_filename': 'mallm-v2/0.mp3'}

In [6]:
import pandas as pd

pd.DataFrame(result).to_parquet('mallm-segment-timestamps.parquet')

In [7]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="mallm-segment-timestamps.parquet",
    path_in_repo="data/mallm_segment-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

mallm-segment-timestamps.parquet:   0%|          | 0.00/439k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/56b4527d6fa3bd78b2eb495d55e7a8e1212f09ca', commit_message='Upload data/mallm_segment-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='56b4527d6fa3bd78b2eb495d55e7a8e1212f09ca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)